<a href="https://colab.research.google.com/github/JhonR-26/Regresion_logistica/blob/main/Actividad%20-%20Regresion%20logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
# Import the packages
import Pkg;
Pkg.add("Lathe")
using Lathe

import Pkg;
Pkg.add("DataFrames")
using DataFrames

import Pkg;
Pkg.add("Plots")
using Plots;

import Pkg;
Pkg.add("GLM")
using GLM

import Pkg;
Pkg.add("StatsBase")
using StatsBase

import Pkg;
Pkg.add("MLBase")
using MLBase

import Pkg;
Pkg.add("ROCAnalysis")
using ROCAnalysis

import Pkg;
Pkg.add("CSV")
using CSV

import Pkg;
Pkg.add("HTTP")
using HTTP

import Pkg;
Pkg.add("Printf")
using Printf

println("Successfully imported the packages!!")

In [ ]:
using CSV
using DataFrames

# Ruta al archivo Excel
ruta_archivo = "/content/Base_de_datos_.xlsx - Hoja 1.csv"

# Leer el archivo Excel y convertirlo en un DataFrame
df = CSV.File(ruta_archivo) |> DataFrame

In [ ]:
# Supongamos que tienes un DataFrame llamado df y deseas convertir la columna 'columna_nombre'
columna_lista = df[!, "clase"]
lista_resultante = Vector(columna_lista)

In [114]:
# Train test split
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df, .75);

In [ ]:
# Train logistic regression model
fm = @formula(clase ~ duracion + paginas + acciones + valor)
logit = glm(fm, train, Binomial(), ProbitLink())

In [116]:
# Coeficientes (B0, B1, B2, B3, B4)
coeficientes = coef(logit)

# Reordenar el número con dos decimales
B0 = @sprintf("%.4f", coeficientes[1])
B1 = @sprintf("%.4f", coeficientes[2])
B2 = @sprintf("%.4f", coeficientes[3])
B3 = @sprintf("%.4f", coeficientes[4])
B4 = @sprintf("%.4f", coeficientes[5])

println("B0 = ", B0, "\n", "B1 = ", B1, "\n", "B2 = ", B2, "\n", "B3 = ", B3, "\n", "B4 = ", B4)

B0 = 0.1600
B1 = 0.0012
B2 = 0.2741
B3 = -0.1509
B4 = 0.0100


In [117]:
# Construir la fórmula de regresión logística
formula_ = "P = 1 / (1 + exp(-(B0 + B1x1 + B2x2 + B3x3 + B4x4)))"

# Reemplazar los valores de los coeficientes en la fórmula
formula_regresion_logistica = replace(formula_, "B0" => B0, "B1" => B1, "B2" => B2, "B3" => B3, "B4" => B4)

println("Fórmula de la regresión logística:")
println(formula_regresion_logistica)

Fórmula de la regresión logística:
P = 1 / (1 + exp(-(0.1600 + 0.0012x1 + 0.2741x2 + -0.1509x3 + 0.0100x4)))


In [ ]:
prediccion = predict(logit, test)

In [ ]:
# Conversion de las probabilidades a clases
evaluacion = [if x < 0.5 0 else 1 end for x in prediccion];

prediccion_df = DataFrame(y_actual = test.clase,
						y_predicted = evaluacion,
						prob_predicted = prediccion);
prediccion_df.correctly_classified = prediccion_df.y_actual .== prediccion_df.y_predicted

In [120]:
# Matriz de confusion
matriz_de_confusion = MLBase.roc(prediccion_df.y_actual, prediccion_df.y_predicted)

ROCNums{Int64}
  p = 15
  n = 30
  tp = 12
  tn = 22
  fp = 8
  fn = 3


In [128]:
# gurdado de los valores de la matriz de confusion
p = matriz_de_confusion.p
n = matriz_de_confusion.n
tp = matriz_de_confusion.tp
tn = matriz_de_confusion.tn
fp = matriz_de_confusion.fp
fn = matriz_de_confusion.fn


# Exactitud, Precision y Sensibilidad del modelo
Exactitud = (tp + tn)/(tp + tn + fp + fn)
Precision = tp/(tp + fn)
Sensibilidad = tp/(tp + fn)
println("Exactitud = ", Exactitud, "\n", "Precision = ", Precision, "\n", "Sensibilidad = ", Sensibilidad)

Exactitud = 0.7555555555555555
Precision = 0.8
Sensibilidad = 0.8
